<a href="https://colab.research.google.com/github/Luisamfer/Ativ_Est_Desc/blob/main/Estat_desc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Definição da população#
#População Acessível: Agricultores Familiares que realizaram o Cadastro Nacional da Agricultura Familiar#
#Amostragem Não Probabilística Intencional: Agricultores da região Nordeste, tendo em vista a finalidade do estudo#
#Variável a ser analisada: Renda #
#Variável Quantitativa Contínua #
# Distribuição de Frequencia:   ABSOLUTA / RELATIVA / FREQ.ACUMULADA
# Medidas de Tendência Central: MÉDIA / MEDIANA / MODA
# Medidas de Dispersão: VARIÂNCIA / DESVIO PADRÃO / AMPLITUDE / INTERVALO INTERQUARTÍLICO
# Medidas de Posição: Quartis / DEcis / Percentis / Quantis
# Medidas de Forma: Curtose / Assimetria
# Medidas de Relação: Covarância / Correlação

In [1]:
# Importando as bibliotecas necessárias para análise dos dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as st

In [12]:
# Importando o arquivo pelo Google Drive
from google.colab import drive

In [25]:
# Carregando arquivo de MEMBROS
drive.mount('/content/drive')

df_membros = pd.read_csv("/content/drive/My Drive/Colab Notebooks/MBA_Est_Desc_dados/MEMBROS.csv", sep=';', encoding='utf-8', dtype={
    "CAF":"string",
    "CODIGO_PESSOA":"string",
    "SEXO":"category",
    "ESCOLARIDADE":"category",
    "MAO_DE_OBRA_FAMILIAR":"category"
    })

# Converter 'DATA_NASCIMENTO' para formato datetime
df_membros["DATA_NASCIMENTO"] = pd.to_datetime(df_membros["DATA_NASCIMENTO"], dayfirst=True, errors='coerce')

# Calcular a idade
hoje = datetime.today()
df_membros["IDADE"] = df_membros["DATA_NASCIMENTO"].apply(lambda x: int(hoje.year - x.year - ((hoje.month, hoje.day) < (x.month, x.day))) if pd.notnull(x) else None)

df_membros.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,CAF,CODIGO_PESSOA,SEXO,DATA_NASCIMENTO,ESCOLARIDADE,MAO_DE_OBRA_FAMILIAR,IDADE
0,AC012023.01.000103697CAF,228471,FEMININO,1950-02-17,ANALFABETO,SIM,75
1,AC012023.01.000103709CAF,228657,MASCULINO,1994-03-01,MEDIO_COMPLETO,SIM,31
2,AC012023.01.000103709CAF,228738,FEMININO,1998-03-01,MEDIO_COMPLETO,NÃO,27
3,AC012023.01.000103723CAF,228694,MASCULINO,1969-06-21,ANALFABETO,SIM,55
4,AC012023.01.000103723CAF,228735,FEMININO,1982-02-28,ANALFABETO,SIM,43


In [27]:
df_membros.info()
df_membros.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4535543 entries, 0 to 4535542
Data columns (total 7 columns):
 #   Column                Dtype         
---  ------                -----         
 0   CAF                   string        
 1   CODIGO_PESSOA         string        
 2   SEXO                  category      
 3   DATA_NASCIMENTO       datetime64[ns]
 4   ESCOLARIDADE          category      
 5   MAO_DE_OBRA_FAMILIAR  category      
 6   IDADE                 int64         
dtypes: category(3), datetime64[ns](1), int64(1), string(2)
memory usage: 151.4 MB


,DATA_NASCIMENTO,IDADE
count,4535543,4.535543e+06
mean,1982-11-09 22:08:21.858498432,4.181955e+01
min,1909-05-19 00:00:00,0.000000e+00
25%,1968-09-11 00:00:00,2.800000e+01
50%,1981-06-23 00:00:00,4.300000e+01
75%,1996-09-19 00:00:00,5.600000e+01
max,2025-02-13 00:00:00,1.150000e+02
std,NaN,1.837491e+01


In [6]:
# Carregando arquivo de ÁREA
drive.mount('/content/drive')

df_area = pd.read_csv("/content/drive/My Drive/Colab Notebooks/MBA_Est_Desc_dados/AREA.csv", sep=';', encoding='utf-8', dtype={
    "CAF":"string",
    "UF":"string",
    "MUNICIPIO":"string",
    "IMOVEL_PRINCIPAL":"category",
    "USO_TERRA":"category"
})

print("\nInformações gerais:")
print(df_area.info())
print("\nDescrição estatística:")
print(df_area.describe())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Informações gerais:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2587032 entries, 0 to 2587031
Data columns (total 5 columns):
 #   Column            Dtype   
---  ------            -----   
 0   CAF               string  
 1   UF                string  
 2   MUNICIPIO         string  
 3   IMOVEL_PRINCIPAL  string  
 4   USO_TERRA         category
dtypes: category(1), string(4)
memory usage: 81.4 MB
None

Descrição estatística:
                             CAF       UF MUNICIPIO IMOVEL_PRINCIPAL  \
count                    2587032  2587032   2587032          2587032   
unique                   2365365       28      5251                2   
top     SE012025.01.002554986CAF       BA  OURICURI              SIM   
freq                          21   458615      8910          2365363   

           USO_TERRA  
count        2587032  
unique            11  
to

In [ ]:
# Carregando arquivo de RENDA
drive.mount('/content/drive')

df_renda = pd.read_csv("/content/drive/My Drive/Colab Notebooks/MBA_Est_Desc_dados/RENDA.csv", sep=';', encoding='utf-8')

print("\nInformações gerais:")
print(df_renda.info())
#print("\nDescrição estatística:")
#print(df_renda.describe())